In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Loading Data 

Here, we'll load the data in data frames of pandas.

In [ ]:
train_csv = '/kaggle/input/Kannada-MNIST/train.csv'
test_csv = '/kaggle/input/Kannada-MNIST/test.csv'
val_csv = '/kaggle/input/Kannada-MNIST/Dig-MNIST.csv'

In [ ]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
valid_df = pd.read_csv(val_csv)

Just checking out how does the data frame looks

In [ ]:
valid_df.head()

# Visualizing Data

Here, we're seeing how many samples are present there for each number of classes. To avoid any bias by the network, we'll balance the data by augumentation in case there are no approximately equal number of samples per class

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(train_df['label'], kde=False)

Looks like there are exactly equal number of samples per class. Thus, we don't need any data augumentation to overcome biasing.

# Formatting Data

In [ ]:
X_train = train_df.drop('label', axis=1).values
y_train = train_df['label'].values

X_val = valid_df.drop('label', axis=1).values
y_val = valid_df['label'].values

### Reshaping data in (28, 28, 1) dims

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28)
X_val = X_val.reshape(X_val.shape[0], 28, 28)

In [ ]:
X_train = np.expand_dims(X_train, axis=3)
X_val = np.expand_dims(X_val, axis=3)
n_classes = 10 # 0 through 9

Converting labels to one-hot encoding

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

### Increasing number of samples
Using ImageDataGenerator here.
Ref: <a href="https://www.kaggle.com/shahules/indian-way-to-learn-cnn"> Link </a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_data_gen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [ ]:
image_data_gen.fit(X_train)

In [ ]:
image_data_gen.fit(X_val)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
img_shape = (28, 28, 1)

# Model Definition

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, UpSampling2D
from tensorflow.keras.optimizers import Adam

Model Ref: <a href="https://www.kaggle.com/shahules/indian-way-to-learn-cnn"> Link </a>

In [ ]:
model = Sequential()

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(BatchNormalization())


model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(3, 3), data_format=None, interpolation='nearest'))

model.add(Conv2D(filters=16, kernel_size=(2, 2), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Flatten())


model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.4))

model.add(Dense(10, activation = "softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

# Training

In [ ]:
model.fit_generator(image_data_gen.flow(X_train, y_train, batch_size=32),
                   epochs=10,
                   validation_data=(image_data_gen.flow(X_val, y_val)))

In [ ]:
pd.DataFrame(model.history.history).plot()

# Testing on Test Dataset

In [ ]:
test_df.head()

In [ ]:
x_test = test_df.drop('id', axis=1).values.reshape(len(test_df), 28, 28, 1)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
submit_df = pd.DataFrame(columns=['id', 'label'])
submit_df.index.name = 'id'

In [ ]:
for index, pred in enumerate(predictions):
    df = pd.DataFrame({
        'id': [index],
        'label': [pred.argmax()]
    })
    submit_df = submit_df.append(df)

In [ ]:
submit_df.head()

In [ ]:
submit_df.to_csv('./submission.csv', index=False)

<a href='./submission.csv'> Download </a>

In [ ]:
model.layers

# Visualizing the layer

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
layers = [layer.output for layer in model.layers]
len(layers)

Choosing any random layer here to viz.
<br>
I chose 19th.

In [ ]:
partial_model = Model(inputs=model.input, outputs=[layers[:19]])

In [ ]:
x_test.shape

In [ ]:
predicted_partial = partial_model.predict(x_test[220].reshape(1, 28, 28, 1))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(predicted_partial[0][0, :, :, 1])
predicted_partial[0].shape

In [ ]:
plt.imshow(x_test[0].reshape(28, 28))
# x_test[0].shape